In [43]:
import pandas as pd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [44]:
df_clean = pd.read_csv('data/df_clean.csv')
display(df_clean)
df_clean.loc[df_clean['bmi'] < 18.5 , 'grade'] = 'underweight'
df_clean.loc[df_clean['bmi'].between(18.5, 25, 'both'), 'grade'] = 'good'
df_clean.loc[df_clean['bmi'].between(25, 30, 'right'), 'grade'] = 'overweight'
df_clean.loc[df_clean['bmi'] > 30 , 'grade'] = 'obesity'
df_clean.drop('bmi',axis=1, inplace=True)

# selection des features et de la cible
X = df_clean.drop('charges', axis=1)

y = df_clean.charges

# Création du train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, train_size=0.85, random_state=42, stratify=X['smoker'])

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1332,50,male,30.970,3,no,northwest,10600.54830
1333,18,female,31.920,0,no,northeast,2205.98080
1334,18,female,36.850,0,no,southeast,1629.83350
1335,21,female,25.800,0,no,southwest,2007.94500


In [45]:
df_clean.columns

Index(['age', 'sex', 'children', 'smoker', 'region', 'charges', 'grade'], dtype='object')

In [46]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures, OrdinalEncoder
from sklearn.linear_model import LinearRegression, Lasso,ElasticNet,Ridge
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.preprocessing import FunctionTransformer


# Séparation des colonnes numériques et catégorielles

cat_col = list(X.select_dtypes(include = 'object').columns)

num_col = list(X.select_dtypes(include = 'number').columns)


preprocessor = make_column_transformer((StandardScaler(),num_col),  
                                       (OneHotEncoder(drop= 'if_binary'),cat_col),
                                       remainder='passthrough'
                                       )

final_preprocessor = make_pipeline(preprocessor,StandardScaler(),PolynomialFeatures(degree=2))

In [48]:
#affichage

tab = final_preprocessor.fit(X_train)

cols = tab.get_feature_names_out()

df_transform = pd.DataFrame(final_preprocessor.fit_transform(X_train), columns=cols)
df_transform.head()

,1,standardscaler__age,standardscaler__children,onehotencoder__sex_male,onehotencoder__smoker_yes,onehotencoder__region_northeast,onehotencoder__region_northwest,onehotencoder__region_southeast,onehotencoder__region_southwest,onehotencoder__grade_good,...,onehotencoder__grade_good^2,onehotencoder__grade_good onehotencoder__grade_obesity,onehotencoder__grade_good onehotencoder__grade_overweight,onehotencoder__grade_good onehotencoder__grade_underweight,onehotencoder__grade_obesity^2,onehotencoder__grade_obesity onehotencoder__grade_overweight,onehotencoder__grade_obesity onehotencoder__grade_underweight,onehotencoder__grade_overweight^2,onehotencoder__grade_overweight onehotencoder__grade_underweight,onehotencoder__grade_underweight^2
0,1.0,-1.508889,2.452595,-1.017763,-0.507965,1.760989,-0.558371,-0.622139,-0.561084,-0.443903,...,0.19705,-0.408572,0.275565,0.056325,0.847154,-0.571371,-0.116788,0.385366,0.078768,0.016100
1,1.0,1.766367,0.768613,-1.017763,1.968638,-0.567863,-0.558371,-0.622139,1.782266,-0.443903,...,0.19705,-0.408572,0.275565,0.056325,0.847154,-0.571371,-0.116788,0.385366,0.078768,0.016100
2,1.0,-1.437688,-0.915369,0.982547,-0.507965,-0.567863,1.790924,-0.622139,-0.561084,-0.443903,...,0.19705,0.482288,-0.715074,0.056325,1.180422,-1.750177,0.137859,2.594937,-0.204399,0.016100
3,1.0,-1.437688,-0.915369,0.982547,-0.507965,-0.567863,1.790924,-0.622139,-0.561084,-0.443903,...,0.19705,0.482288,0.275565,-3.498423,1.180422,0.674459,-8.562554,0.385366,-4.892392,62.111111
4,1.0,-0.583273,1.610604,0.982547,1.968638,-0.567863,-0.558371,-0.622139,1.782266,-0.443903,...,0.19705,0.482288,-0.715074,0.056325,1.180422,-1.750177,0.137859,2.594937,-0.204399,0.016100


In [39]:
final_preprocessor

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('standardscaler',
                                                  StandardScaler(),
                                                  ['age', 'children']),
                                                 ('onehotencoder',
                                                  OneHotEncoder(drop='if_binary'),
                                                  ['sex', 'smoker', 'region',
                                                   'grade'])])),
                ('standardscaler', StandardScaler()),
                ('polynomialfeatures', PolynomialFeatures())])

In [40]:
from sklearn.model_selection import GridSearchCV

model= make_pipeline(final_preprocessor, ElasticNet(max_iter=100000,random_state=42))

# Définir les valeurs à tester pour alpha et l1_ratio

param_grid = {'elasticnet__alpha': np.arange(149,152,0.1),   
              'elasticnet__l1_ratio':[1]}      


# creation de l'objet GridSearch

grid_search = GridSearchCV(model,param_grid, cv=5)

# recherche des hyperparamètres et Afficher les meilleures valeurs trouvées

grid_search.fit(X_train, y_train)

print("Meilleurs paramètres : ",grid_search.best_params_)

# Utiliser le modèle avec les meilleures valeurs
best_model = grid_search.best_estimator_

best_model.score(X_test, y_test)

print("Score : ", best_model.score(X_test, y_test))

# calcul RSME

rmse= np.sqrt(mean_squared_error(y_test,best_model.predict(X_test)))
print("RSME: ",rmse)


Meilleurs paramètres :  {'elasticnet__alpha': 151.09999999999988, 'elasticnet__l1_ratio': 1}
Score :  0.9219211898204945
RSME:  3363.7556918045966


In [42]:
import pickle

with open ('best_model.pkl', 'wb') as model_file:
    pickle.dump(best_model,model_file)

In [29]:
model= make_pipeline(final_preprocessor, ElasticNet(max_iter=1000,random_state=42, l1_ratio=1, alpha=55.55555555555556))

model.fit(X_train,y_train)

model.score(X_test,y_test)

0.92143743028362

## Regression Linéaire 

In [30]:
# regression lineaire

model = make_pipeline(final_preprocessor,LinearRegression())
model.fit(X_train,y_train)
model.score(X_train,y_train)
print("R2 (coefficient de détermination): ",model.score(X_test,y_test))


R2 (coefficient de détermination):  0.9178290803565263


## Regression Lasso

In [31]:
model= make_pipeline(final_preprocessor,Lasso(max_iter=1000000))

param_grid={'lasso__alpha':np.arange(0.1,100,10)}

grid_search = GridSearchCV(model,param_grid, cv=5)

grid_search.fit(X_train, y_train)

print("Meilleurs paramètres : ",grid_search.best_params_)

# Utiliser le modèle avec les meilleures valeurs
best_model = grid_search.best_estimator_

best_model.score(X_test, y_test)


Meilleurs paramètres :  {'lasso__alpha': 90.1}


0.9214908596498723

## Regression Ridge

In [32]:
model= make_pipeline(final_preprocessor,Ridge(max_iter=1000000))
model.fit(X_train,y_train)
model.score(X_test,y_test)

0.9176473243203678